In [4]:
import json
import pytz
from send_file import send_file
from datetime import datetime
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options

def get_articles():

    with open ("get_headers_news_local.json", "r") as dataset:
        json_data = json.load(dataset)

    # Chromeオプションを設定
    chrome_options = Options()
    chrome_options.add_argument("--headless")  # ヘッドレスモードを有効にする

    # Chromeドライバのインスタンスを作成
    driver = webdriver.Chrome(options=chrome_options)

    # 各URLに対して処理を実行
    for target in json_data:
        try:

            driver.get(target["url"])

            # WebDriverWaitを使って最大10秒間待機
            element = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, target["selector"]))
            )
            
            # 前回の見出しを保存しておく
            target["get_title_before"] = target["get_title"]
            target["get_title"] = element.text

            # 結果を表示
            print(
                f"URL: {target['url']}\n" + 
                target["head"] +
                "\nNew title\n" +
                str(target["get_title"]) + 
                "\nBefore title\n" +
                str(target["get_title_before"])
            )

            # 差分のが出たかを見て、値として保存
            if target["get_title_before"] != target["get_title"]:
                print("Update\n")
                target["update"] = True
            else:
                print("No update\n")        
                target["update"] = False

        except Exception as e:
            print(f"Error occurred while processing URL: {target['url']}")
            print(e)

    # 全ての処理が終わった後、ドライバを終了
    driver.quit()

    # 処理の結果を保存
    with open("./get_headers_news_local.json", "w", encoding="utf-8") as json_file:
        json.dump(json_data, json_file, ensure_ascii=False, indent=4)

    # スクレイピングの結果から、アップデートがあったものだけ配列に追加
    updates = []
    for web in json_data:
        if web["update"] == True:
            updates.append(web)
            
    print(f"UPDATE: {updates}")
        
    # 一つ以上アップデートがある場合、ファイルを上書きして保存
    if 0 < len(updates):
        print("get updates")
        japan_tz = pytz.timezone('Asia/Tokyo')
        current_time_japan = datetime.now(japan_tz)
        formatted_time = current_time_japan.strftime('%Y/%m/%d %H:%M')

        new_object = {
            "date": formatted_time,
            "updates": updates
        }

        # 現状を読み込んで変数に格納
        with open ("get_headers_news_local_update.json", "r") as update:
            update_json = json.load(update)
            update_json.append(new_object)

        # 更新分のオブジェクトを配列を追加して、jsonファイルとして保存
        with open ("get_headers_news_local_update.json", "w") as update:
            json.dump(update_json, update, ensure_ascii=False, indent=4)
        
        # ファイルをstorageに送信する
        send_file("test send")

    else:
        print("no update")

get_articles()

URL: https://www.nikkei.com/news/category/
日経電子版 速報
New title
NTT｢法整備に協力｣　自民党の廃止提言で
Before title
富士フイルム「チェキ」、24年度に売上高1500億円に
Update

URL: https://www3.nhk.or.jp/news/catnew.html
NHKニュース 速報・新着一覧
New title
【随時更新】イスラエル軍 南部への空爆強化 地上部隊を展開
Before title
【随時更新】イスラエル軍 南部への空爆強化 地上部隊を展開
No update

URL: https://news.yahoo.co.jp/flash
Yahoo!ニュース 速報
New title
緊迫のガザ南部「今も爆撃、家が揺れた」　避難住民に危機感
Before title
安倍派がパーティー券“専用口座”つくり管理か　国会では”政治とカネ”追及続く
Update

UPDATE: [{'head': '日経電子版 速報', 'media': 'NIKKEI', 'url': 'https://www.nikkei.com/news/category/', 'selector': '#CONTENTS_MAIN > div:nth-child(1) > h3 > a > span', 'get_title': 'NTT｢法整備に協力｣\u3000自民党の廃止提言で', 'get_title_before': '富士フイルム「チェキ」、24年度に売上高1500億円に', 'update': True}, {'head': 'Yahoo!ニュース 速報', 'media': 'Yahoo!', 'url': 'https://news.yahoo.co.jp/flash', 'selector': '#contentsWrap > div:nth-child(3) > div.sc-dMCwnC.lipeMg > div > a > p', 'get_title': '緊迫のガザ南部「今も爆撃、家が揺れた」\u3000避難住民に危機感', 'get_title_before': '安倍派がパーティー券“専用口座”つくり管理か\u3000国会では”政治とカネ”追及続く', 'u